In [5]:
import psycopg2
import numpy as np
import pandas as pd
import folium
import json
# 3rd party modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import shapely
# needs 'descartes'
import matplotlib.pyplot as plt
import re
import fileinput

In [6]:
conn=psycopg2.connect(dbname= '',host='',
                      port='',
                      user='',password='' )

In [ ]:
# OPTION 1---for 25 store using SQL query

req_location_data_with_pin=pd.read_sql_query("""SELECT
location_code,
name,
lower(city) as city,
pincode,
format,
latitude,
longitude
from table
 """,conn)

In [ ]:
req_location_data_with_pin.dropna(how='any',inplace=True)

In [ ]:
req_location_data_with_pin.isnull().sum()

In [ ]:
req_location_data_with_pin.location_code.unique()

In [ ]:
###converting to float
req_location_data_with_pin["longitude"] = pd.to_numeric(req_location_data_with_pin["longitude"], downcast='float')#, errors='coerce')
req_location_data_with_pin["latitude"] = req_location_data_with_pin["latitude"].apply(pd.to_numeric, downcast='float', errors='coerce')

req_location_data_with_pin["location_code"] = req_location_data_with_pin["location_code"].astype(int)

In [ ]:
## removing string na
req_location_data_with_pin.loc[req_location_data_with_pin.pincode.isna(),"pincode"]="NA"
req_location_data_with_pin.loc[req_location_data_with_pin.latitude.isna(),"latitude"]="NA"
req_location_data_with_pin.loc[req_location_data_with_pin.longitude.isna(),"longitude"]="NA"
req_location_data_with_pin.dropna(inplace=True)

In [ ]:
pincode_data_boundary = json.load(open("India_Pincode_Boundary.geojson"))

#pincode_data_boundary

In [ ]:
city  = "Cochin"

In [ ]:
filtered_data = []
filtered_pincodes=set(req_location_data_with_pin[req_location_data_with_pin.city==city.lower()].pincode.astype(str).unique())

filtered_pincodes = set([pincode if len(pincode.split("."))==1 else pincode.split(".")[0]  for pincode in filtered_pincodes])
for i in range(len(pincode_data_boundary["features"])):
    if pincode_data_boundary["features"][i]["properties"]["pincode"] in filtered_pincodes :
        filtered_data.append(pincode_data_boundary["features"][i])

In [ ]:
pincode_data_boundary_bangalore = {"type":"FeatureCollection",
"name":"India_Pincode_02122020",
"crs":{'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
"features": filtered_data  }

In [ ]:
pd.DataFrame.from_dict(pincode_data_boundary["features"])

In [ ]:
## converting to polygon format and finding cetroid for each boundary
ls= [list(pincode_data_boundary["features"][i]["properties"].values())+pincode_data_boundary["features"][i]["geometry"]["coordinates"][0] for i in range(len(pincode_data_boundary["features"]))]

geojson_to_dataframe = pd.DataFrame(ls)

geojson_to_dataframe["geometry"] = geojson_to_dataframe[5].apply(Polygon)

geojson_to_dataframe = geojson_to_dataframe.drop(columns=[5,6,7,8,9])

geojson_to_dataframe.columns = ["pincode","state","district","officename","officetype","geometry"]

geojson_to_geodataframe = gpd.GeoDataFrame(geojson_to_dataframe)

geojson_to_geodataframe["centroid"] = geojson_to_geodataframe.geometry.centroid

In [ ]:
geojson_to_geodataframe.head()

In [ ]:
pincodes_from_lat_long = geojson_to_geodataframe.copy()
pincodes_from_lat_long.pincode = pincodes_from_lat_long.pincode.astype(str)
pincodes_from_lat_long = pincodes_from_lat_long[pincodes_from_lat_long.pincode.isin(filtered_pincodes)]

In [ ]:
##for focusing on that location
my_map = folium.Map(location = [req_location_data_with_pin[req_location_data_with_pin.city==city.lower()].latitude.mean(),
                                req_location_data_with_pin[req_location_data_with_pin.city==city.lower()].longitude.mean()],
                                        zoom_start = 11,control_scale=True)

In [ ]:
# for potting the pincode boundaries
folium.GeoJson(
            pincode_data_boundary_bangalore,
            style_function=lambda feature: {
                "fillColor": "yellow",
                'color': "blue",
                'weight': 1,
                "fill_opacity": 0    
                #,'fillOpacity': fill_opacity 
            }, name = "store_locations").add_to(my_map)

In [ ]:
# for marking the stores with popup
for indice,row in req_location_data_with_pin[req_location_data_with_pin.city==city.lower()].iterrows():
    html=f"""
        <h1> {row["name"]}</h1>
        <p>Properties</p>
        <ul>
            <li>Location_code: {row.location_code}</li>
            <li>City: {row.city}</li>
            <li>Pincode:{row.pincode}</li>
            <li>latitude:{row.latitude}</li>
            <li>longitude:{row.longitude}</li>
            <li>format:{row.format}</li>
        </ul>
        </p>
        """
    iframe = folium.IFrame(html=html, width=300, height=300)
    popup = folium.Popup(iframe, max_width=3650)
    folium.Marker([row.latitude, row.longitude],
               popup = popup).add_to(my_map)
    
    
# for indice,row in req_location_data_with_pin[req_location_data_with_pin.city=="gajuwaka"].iterrows():
#     html=f"""
#         <h1> {row["name"]}</h1>
#         <p>Properties</p>
#         <ul>
#             <li>Location_code: {row.location_code}</li>
#             <li>City: {row.city}</li>
#             <li>Pincode:{row.pincode}</li>
#             <li>latitude:{row.latitude}</li>
#             <li>longitude:{row.longitude}</li>
#             <li>format:{row.format}</li>
#         </ul>
#         </p>
#         """
#     iframe = folium.IFrame(html=html, width=300, height=300)
#     popup = folium.Popup(iframe, max_width=3650)
#     folium.Marker([row.latitude, row.longitude],
#                popup = popup,
#                  icon=folium.Icon(color="red")).add_to(my_map)
    



# whats app data
# for indice,row in req_location_data_with_pin[req_location_data_with_pin.city==city.lower()].iterrows():
#     html=f"""
#         <h1> {row["name"]}</h1>
#         <p>Properties</p>
#         <ul>
#             <li>Location_code: {row.location_code}</li>
#             <li>City: {row.revised_city}</li>
#             <li>Pincode:{row.pincode}</li>
#             <li>latitude:{row.store_latitude}</li>
#             <li>longitude:{row.store_longitude}</li>
#         </ul>
#         </p>
#         """
#     iframe = folium.IFrame(html=html, width=300, height=300)
#     popup = folium.Popup(iframe, max_width=3650)
#     folium.Marker([row.store_latitude, row.store_longitude],
#                popup = popup).add_to(my_map)


# for plotting pincode names
for indice,row in pincodes_from_lat_long.iterrows():
    folium.Marker(location = [row.centroid.y, row.centroid.x],
                  popup =row.pincode,
                  icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: red">{row.pincode}</div>""")
                 ).add_child(folium.Popup(row.pincode)).add_to(my_map)
    

# for indice,row in req_location_data_with_pin[req_location_data_with_pin.city==city.lower()].iterrows():
#     folium.Circle([row.latitude,row.longitude],
#                  radius=10000,
#                  fill_color="green",
#                  stroke=False,
#                  fillopacity=1).add_to(my_map)

In [ ]:
my_map.save("{city}_map.html".format(city=city))

In [ ]:
# for pop up with mouse hovering

with open("{city}_map.html".format(city=city)) as inf:
    txt = inf.read()

#Find all the markers names given by folium
markers = re.findall(r'\bmarker_\w+', txt)
markers = list(set(markers))

for marker in markers:
    for linenum,line in enumerate( fileinput.FileInput("{city}_map.html".format(city=city),inplace=1) ):
        pattern = marker + ".bindPopup"
        pattern2 = marker + ".on('mouseover', function (e) {this.openPopup();});"
        pattern3 = marker + ".on('mouseout', function (e) {this.closePopup();});"

        if pattern in line:
            print(line.rstrip())
            print(pattern2)
            print(pattern3)
        else:
            print(line.rstrip())

In [ ]:
# for pop up with mouse hovering

with open("{city}_map.html".format(city=city)) as inf:
    txt = inf.read()

#Find all the markers names given by folium
markers = re.findall(r'\bmarker_\w+', txt)
markers = list(set(markers))

for marker in markers:
    for linenum,line in enumerate( fileinput.FileInput("{city}_map.html".format(city=city),inplace=1) ):
        pattern = marker + ".bindPopup"
        pattern2 = marker + ".on('mouseover', function (e) {this.openPopup();});"
        pattern3 = marker + ".on('mouseout', function (e) {this.closePopup();});"

        if pattern in line:
            print(line.rstrip())
            print(pattern2)
            print(pattern3)
        else:
            print(line.rstrip())